In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import h5py
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        residual = x
        out = self.conv_block(x)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(EncoderBlock, self).__init__()
        self.res_block = ResidualBlock(in_channels, out_channels, stride)
        self.pool = nn.MaxPool2d(2) if stride == 1 else nn.Identity()
        
    def forward(self, x):
        x = self.res_block(x)
        pool = self.pool(x)
        return x, pool

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.res_block = ResidualBlock(out_channels * 2, out_channels)  # *2 for concatenation
        
    def forward(self, x, skip):
        x = self.up(x)
        # Handle size mismatches for concatenation
        if x.size()[2:] != skip.size()[2:]:
            x = F.interpolate(x, size=skip.size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x, skip], dim=1)
        x = self.res_block(x)
        return x

class ResNetUNet(nn.Module):
    def __init__(self, n_channels, n_classes, filters=(64, 128, 256, 512, 1024)):
        super(ResNetUNet, self).__init__()
        
        # Initial convolution
        self.initial = nn.Sequential(
            nn.Conv2d(n_channels, filters[0], kernel_size=7, stride=1, padding=3, bias=False),
            nn.BatchNorm2d(filters[0]),
            nn.ReLU(inplace=True)
        )
        
        # Encoder
        self.enc1 = EncoderBlock(filters[0], filters[0])
        self.enc2 = EncoderBlock(filters[0], filters[1])
        self.enc3 = EncoderBlock(filters[1], filters[2])
        self.enc4 = EncoderBlock(filters[2], filters[3])
        
        # Bottleneck
        self.bottleneck = ResidualBlock(filters[3], filters[4])
        
        # Decoder
        self.dec4 = DecoderBlock(filters[4], filters[3])
        self.dec3 = DecoderBlock(filters[3], filters[2])
        self.dec2 = DecoderBlock(filters[2], filters[1])
        self.dec1 = DecoderBlock(filters[1], filters[0])
        
        # Output convolution
        self.output = nn.Conv2d(filters[0], n_classes, kernel_size=1)
        
    def forward(self, x):
        # Initial convolution
        x = self.initial(x)
        
        # Encoder
        skip1, x = self.enc1(x)
        skip2, x = self.enc2(x)
        skip3, x = self.enc3(x)
        skip4, x = self.enc4(x)
        
        # Bottleneck
        x = self.bottleneck(x)
        
        # Decoder
        x = self.dec4(x, skip4)
        x = self.dec3(x, skip3)
        x = self.dec2(x, skip2)
        x = self.dec1(x, skip1)
        
        # Output
        output = self.output(x)
        
        return output



In [2]:
import os
import random
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchvision import transforms

# ---------------- Dataset Class ----------------
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, image_filenames=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform

        if image_filenames is None:
            self.images = [img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')]
        else:
            self.images = image_filenames

        self.image_paths, self.mask_paths = [], []

        for img_name in self.images:
            img_path = os.path.join(self.image_dir, img_name)
            base_name = os.path.splitext(img_name)[0]

            mask_path = None
            for suffix in ['_annotation', '_Annotation']:
                for ext in ['.png', '.jpg', '.jpeg', '.tif', '.bmp']:
                    potential_path = os.path.join(self.mask_dir, base_name + suffix + ext)
                    if os.path.exists(potential_path):
                        mask_path = potential_path
                        break
                if mask_path:
                    break

            if os.path.exists(img_path) and mask_path:
                self.image_paths.append(img_path)
                self.mask_paths.append(mask_path)
            else:
                print(f"Warning: Skipping {img_name} - corresponding mask not found")

        print(f"Found {len(self.image_paths)} valid image-mask pairs out of {len(self.images)} images")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        mask = Image.open(self.mask_paths[idx]).convert('L')

        image = np.array(image)
        mask = np.array(mask)

        # Optional: ensure size match before albumentations (helps prevent mismatch bugs)
        if image.shape[:2] != mask.shape[:2]:
            min_height = min(image.shape[0], mask.shape[0])
            min_width = min(image.shape[1], mask.shape[1])
            image = image[:min_height, :min_width]
            mask = mask[:min_height, :min_width]

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.unsqueeze(0).float() / 255.


# ---------------- Transforms ----------------
def get_transforms(train=True, target_size=(256, 256)):
    try:
        from albumentations import Compose, HorizontalFlip, VerticalFlip, Normalize, RandomRotate90, Resize
        from albumentations.pytorch import ToTensorV2

        if train:
            return Compose([
                Resize(*target_size),
                HorizontalFlip(p=0.5),
                VerticalFlip(p=0.5),
                RandomRotate90(p=0.5),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
        else:
            return Compose([
                Resize(*target_size),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])
    except ImportError:
        raise ImportError("Albumentations is not installed. Please install it for transformations.")


# ---------------- Train/Val Dataset Split ----------------
def create_train_val_datasets(image_dir, mask_dir, val_split=0.2, random_state=42, target_size=(512, 512)):
    image_filenames = [img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')]

    train_filenames, val_filenames = train_test_split(image_filenames, test_size=val_split, random_state=random_state)

    train_dataset = SegmentationDataset(
        image_dir, mask_dir,
        transform=get_transforms(train=True, target_size=target_size),
        image_filenames=train_filenames
    )

    val_dataset = SegmentationDataset(
        image_dir, mask_dir,
        transform=get_transforms(train=False, target_size=target_size),
        image_filenames=val_filenames
    )

    return train_dataset, val_dataset


# ---------------- Loss Function ----------------
class DiceBCELoss(nn.Module):
    def __init__(self):
        super(DiceBCELoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, inputs, targets, smooth=1):
        inputs_sigmoid = torch.sigmoid(inputs)
        bce = self.bce(inputs, targets)
        intersection = (inputs_sigmoid * targets).sum()
        dice = (2.*intersection + smooth)/(inputs_sigmoid.sum() + targets.sum() + smooth)
        return 1 - dice + bce


# ---------------- Metrics ----------------
def dice_coeff(preds, targets, smooth=1.):
    preds = (torch.sigmoid(preds) > 0.5).float().view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    return ((2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)).item()

def iou_score(preds, targets, smooth=1.):
    preds = (torch.sigmoid(preds) > 0.5).float().view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def accuracy_score(preds, targets):
    preds = (torch.sigmoid(preds) > 0.5).float()
    return (preds == targets).float().mean().item()

def precision_recall_f1(preds, targets, smooth=1e-6):
    preds = (torch.sigmoid(preds) > 0.5).float()
    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()
    precision = tp / (tp + fp + smooth)
    recall = tp / (tp + fn + smooth)
    f1 = 2 * precision * recall / (precision + recall + smooth)
    return precision, recall, f1


# ---------------- Training ----------------
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for images, masks in tqdm(dataloader):
        images, masks = images.to(device), masks.to(device)
        preds = model(images)
        loss = criterion(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)


# ---------------- Validation ----------------
def validate(model, dataloader, criterion, device):
    model.eval()
    val_loss = dice_total = iou_total = acc_total = precision_total = recall_total = f1_total = 0
    with torch.no_grad():
        for images, masks in tqdm(dataloader):
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            masks = masks.squeeze(1) if masks.ndim > 4 else masks

            loss = criterion(preds, masks)
            val_loss += loss.item()
            dice_total += dice_coeff(preds, masks)
            iou_total += iou_score(preds, masks)
            acc_total += accuracy_score(preds, masks)
            p, r, f1 = precision_recall_f1(preds, masks)
            precision_total += p
            recall_total += r
            f1_total += f1

    n = len(dataloader)
    return {
        'val_loss': val_loss / n,
        'dice': dice_total / n,
        'iou': iou_total / n,
        'accuracy': acc_total / n,
        'precision': precision_total / n,
        'recall': recall_total / n,
        'f1': f1_total / n
    }


# ---------------- Save Predictions ----------------
def save_predictions(model, dataloader, save_dir, device):
    model.eval()
    os.makedirs(save_dir, exist_ok=True)
    with torch.no_grad():
        for idx, (images, _) in enumerate(dataloader):
            images = images.to(device)
            preds = model(images)
            preds = (torch.sigmoid(preds) > 0.5).float()

            for i in range(images.size(0)):
                pred_np = preds[i].squeeze().cpu().numpy() * 255
                pred_img = Image.fromarray(pred_np.astype(np.uint8))
                pred_img.save(os.path.join(save_dir, f"pred_{idx*images.size(0)+i}.png"))


# ---------------- Train Model with Enhanced Weight Saving ----------------
def train_model(model, train_loader, val_loader, criterion, optimizer, device, 
                epochs=10, save_dir="weights", save_best=True, save_last=True, save_interval=5):
    """
    Train the model with enhanced weight saving options:
    - save_best: Save the best model based on validation dice coefficient
    - save_last: Save the model after the last epoch
    - save_interval: Save the model every 'save_interval' epochs
    """
    os.makedirs(save_dir, exist_ok=True)
    best_dice = 0
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        metrics = validate(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {metrics['val_loss']:.4f}, Dice: {metrics['dice']:.4f}, IoU: {metrics['iou']:.4f}, " +
              f"Acc: {metrics['accuracy']:.4f}, Prec: {metrics['precision']:.4f}, " +
              f"Recall: {metrics['recall']:.4f}, F1: {metrics['f1']:.4f}")

        # Save model at regular intervals
        if save_interval > 0 and (epoch + 1) % save_interval == 0:
            interval_save_path = os.path.join(save_dir, f"unetpp_epoch_{epoch+1}.pth")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_metrics': metrics
            }, interval_save_path)
            print(f"✅ Saved checkpoint at epoch {epoch+1}")

        # Save best model based on validation dice
        if save_best and metrics['dice'] > best_dice:
            best_dice = metrics['dice']
            best_save_path = os.path.join(save_dir, "best_unetpp.pth")
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_dice': best_dice,
                'val_metrics': metrics
            }, best_save_path)
            print(f"✅ Saved best model with Dice: {best_dice:.4f}")
    
    # Save model after the last epoch
    if save_last:
        last_save_path = os.path.join(save_dir, "last_unetpp.pth")
        torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_metrics': metrics
        }, last_save_path)
        print(f"✅ Saved model after last epoch")
    
    return best_dice

# ---------------- Load Model Weights ----------------
def load_model(model, weights_path, device):
    """
    Load model weights from a checkpoint
    """
    checkpoint = torch.load(weights_path, map_location=device)
    
    # Check if the checkpoint contains the full dictionary or just the state_dict
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Loaded model from epoch {checkpoint['epoch']} with Dice score: {checkpoint.get('best_dice', 'N/A')}")
    else:
        model.load_state_dict(checkpoint)
        print(f"✅ Loaded model weights")
    
    return model

# ---------------- Main ----------------
if __name__ == '__main__':
    # Create directories for weights and predictions
    os.makedirs("weights", exist_ok=True)
    os.makedirs("predictions", exist_ok=True)
    
    # Set paths to your single image and mask folders
    image_dir = './data/images'
    mask_dir = './data/masks'

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize model
    model = ResNetUNet(n_channels=3, n_classes=1).to(device)
    criterion = DiceBCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    # Define image target size for resizing
    target_size = (256, 256)  # Common size for segmentation tasks
    
    # Split the dataset and create data loaders with resized images
    train_dataset, val_dataset = create_train_val_datasets(
        image_dir, 
        mask_dir, 
        val_split=0.2,  # 20% for validation
        random_state=42,  # For reproducibility
        target_size=target_size  # Target size for resizing
    )

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)
    
    # Train model with enhanced weight saving
    best_dice = train_model(
        model, 
        train_loader, 
        val_loader, 
        criterion, 
        optimizer, 
        device, 
        epochs=30, 
        save_dir="weights", 
        save_best=True, 
        save_last=True, 
        save_interval=5
    )
    
    # Load best model for predictions
    model = load_model(model, "weights/best_unetpp.pth", device)
    
    # Generate and save predictions
    save_predictions(model, train_loader, save_dir="predictions", device=device)
    save_predictions(model, val_loader, save_dir="predictions", device=device)
    
    print("Training and prediction complete!")

Using device: cuda


C:\Users\Asus\anaconda3\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Found 799 valid image-mask pairs out of 799 images
Found 200 valid image-mask pairs out of 200 images

Epoch 1/30


100%|██████████| 50/50 [02:19<00:00,  2.78s/it]


Train Loss: 0.4007
Val Loss: 0.2187, Dice: 0.9306, IoU: 0.8728, Acc: 0.9608, Prec: 0.9284, Recall: 0.9354, F1: 0.9306
✅ Saved best model with Dice: 0.9306

Epoch 2/30


...

Train Loss: 0.0690
Val Loss: 0.0862, Dice: 0.9683, IoU: 0.9394, Acc: 0.9822, Prec: 0.9731, Recall: 0.9643, F1: 0.9683

Epoch 30/30


100%|██████████| 50/50 [02:39<00:00,  3.19s/it]


Train Loss: 0.0703
Val Loss: 0.0855, Dice: 0.9689, IoU: 0.9405, Acc: 0.9826, Prec: 0.9718, Recall: 0.9667, F1: 0.9689
✅ Saved checkpoint at epoch 30
✅ Saved model after last epoch


C:\Users\Asus\AppData\Local\Temp\ipykernel_25024\2414175313.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location=device)


✅ Loaded model from epoch 25 with Dice score: 0.9695325672626496
Training and prediction complete!
